In [1]:
import pandas as pd
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.schema import Document

class CustomOutputParser:
    def parse(self, response: str):
        # Custom parsing logic to get the relevant response
        # Assuming the response has a specific format
        # Adjust the parsing logic based on your LLM's response format
        parts = response.split('[/INST]')
        if len(parts) > 1:
            return parts[-1].strip()
        return response.strip()

class LLMRAGModel:
    def __init__(self, llm_name="NousResearch/Llama-2-7b-chat-hf"):
        self.load_model(llm_name)
        self.setup_pipeline()
        self.retriever = self.buildRetrieval()  # Build the retriever once during initialization

    def load_model(self, model_name):
        # Load LLM with quantization configuration
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map='auto',
            torch_dtype=torch.float16,
            quantization_config=bnb_config
        )

    def setup_pipeline(self):
        # Set up the text generation pipeline
        self.llmPipeline = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            torch_dtype=torch.float16,
            device_map="auto",
            max_new_tokens=400,
            do_sample=True,
            top_k=50,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id
        )

        self.llm = HuggingFacePipeline(
            pipeline=self.llmPipeline, 
            model_kwargs={'temperature': 0.1, 'max_length': 350, 'top_k': 10}
        )

    def generate_text(self, prompt):
        # Generate text using the pipeline
        return self.llmPipeline(prompt)

    def getPromptFromTemplate(self):
        system_prompt = """You are PropertyPal, a specialized assistant designed to provide property recommendations based exclusively on the data given to you. Your responses should be based solely on the user's specific queries regarding property details and should reflect only the data available in the dataset.

1. If the user asks for property recommendations, provide the following details for each property that matches their criteria:
   - Location
   - Number of bedrooms
   - Number of bathrooms
   - Property size
   - Price

2. Do not include any additional details such as property IDs, extra information, or suggestions beyond what is present in the dataset.

3. If the user's input is a generic greeting or does not pertain to property recommendations, respond with a neutral message indicating that you can help with property queries and ask them to specify their requirements.

Your responses should be concise and focused on the data provided, with no additional or extraneous details."""

        B_INST, E_INST = "[INST]", "[/INST]"
        B_SYS, E_SYS = " << SYS>>\n", "\n << /SYS>>\n\n"

        SYSTEM_PROMPT1 = B_SYS + system_prompt + E_SYS

        instruction = """
        History: {history} \n
        Context: {context} \n
        User: {question}"""

        prompt_template = B_INST + SYSTEM_PROMPT1 + instruction + E_INST

        prompt = PromptTemplate(input_variables=["history", "question", "context"], template=prompt_template)

        return prompt

    def buildRetrieval(self, model_name="sentence-transformers/all-MiniLM-L6-v2", csv_file=None):
        if csv_file is None:
            csv_file = r"C:\Users\Turabi\OneDrive\Desktop\PropertyPal\PropertyPal\properties.csv"

        # Load the CSV file into a DataFrame
        df = pd.read_csv(csv_file)

        # Create documents from the DataFrame
        documents = df.apply(lambda row: Document(
            page_content=f"Property is located in {row['location']}. The price of the property is {row['price']}. It has {row['bedrooms']} bedrooms and {row['bathrooms']} bathrooms. The size of the property is {row['size']} square feet.",
            metadata=row.to_dict()
        ), axis=1).tolist()

        # Extract texts for embedding
        texts = [doc.page_content for doc in documents]

        # Initialize embeddings and vector store
        embeddings = HuggingFaceEmbeddings(model_name=model_name)
        text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=100, separator="feet.")
        texts = text_splitter.split_documents([Document(page_content=text) for text in texts])

        db = FAISS.from_documents(texts, embeddings)
        retriever = db.as_retriever()

        return retriever

    def getNewChain(self):
        prompt = self.getPromptFromTemplate()
        memory = ConversationBufferMemory(input_key="question", memory_key="history", max_len=5)
        llm_chain = LLMChain(prompt=prompt, llm=self.llm, verbose=True, memory=memory)
        rag_chain = (
            {"context": self.retriever, "question": lambda x: x}
            | llm_chain
        )
        return rag_chain

model_instance = LLMRAGModel()

# Function to get answers from the chain
def getAnswer(chain, question):
    # Invoke the chain and get the response
    response = chain.invoke(question)
    # Assuming the response is a dictionary and `text` key contains the response text
    response_text = response.get('text', 'No response text found')
    # Use the custom output parser to clean up the response
    parser = CustomOutputParser()
    return parser.parse(response_text)


# Create a new chain using the model instance
chain = model_instance.getNewChain()

# Example usage
# answer = getAnswer(chain, "I'm looking for a property in Navy Housing Scheme Karsaz, Karachi which should have 5 bedrooms and price should be 102500000")
# print(answer)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\Turabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 0.3. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  warn_deprecated(
C:\Users\Turabi\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\Turabi\AppData\L

In [2]:
import anvil.server

# Connect to the Anvil server
anvil.server.connect("server_GHCXZYPN7FYGDDV2IRDBJFIG-5EMKN7KUZ3BZKLXO")

# Define your function to handle bot response
@anvil.server.callable
def get_bot_response(prompt):
    # Use the getAnswer function to get the response from the model
    response_text = getAnswer(chain, prompt)
    return response_text

# Keep the connection alive
# Uncomment if needed
# anvil.server.wait_forever()


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Debug for dnntrb@gmail.com" as SERVER


> Entering new LLMChain chain...
Prompt after formatting:
[INST] << SYS>>
You are PropertyPal, a specialized assistant designed to provide property recommendations based exclusively on the data given to you. Your responses should be based solely on the user's specific queries regarding property details and should reflect only the data available in the dataset.

1. If the user asks for property recommendations, provide the following details for each property that matches their criteria:
   - Location
   - Number of bedrooms
   - Number of bathrooms
   - Property size
   - Price

2. Do not include any additional details such as property IDs, extra information, or suggestions beyond what is present in the dataset.

3. If the user's input is a generic greeting or does not pertain to property recommendations, respond with a neutral message indicating that you can help with prope